<a href="https://colab.research.google.com/github/dongjaeseo/colab/blob/main/lung_cancer_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import bisect

In [ ]:
class mask_image():
    def __init__(self, image):
        self.image = image
        self.boundary = self.find_boundary()
        self.cg = self.centroid()
        self.theta_dict = self.theta()
        self.theta_radial = self.find_dist()

    def centroid(self):
        row_sum = 0
        column_sum = 0
        count = 0
        for i, row in enumerate(self.image):
            for j, pixel in enumerate(row):
                if pixel == 1:
                    row_sum += i
                    column_sum += j
                    count += 1

        if count == 0:
            raise TypeError("Wrong image input: Only masked image of 0 and 1 is allowed")

        return column_sum/count, row_sum/count

    def find_boundary(self):
        boundary = []
        for i, row in enumerate(self.image):
            for j, pixel in enumerate(row):

                if i != 0:
                    if self.image[i-1][j] < pixel:
                        boundary.append((j,i-0.5))
                
                if j != len(self.image[0])-1:
                    if self.image[i][j+1] < pixel:
                        boundary.append((j+0.5, i))
                
                if i != len(self.image)-1:
                    if self.image[i+1][j] < pixel:
                        boundary.append((j,i+0.5))

                if j != 0:
                    if self.image[i][j-1] < pixel:
                        boundary.append((j-0.5, i))

        return boundary
    
    # Represent angle between [0, 2pi]
    def theta(self):
        theta_boundary = {}
        for bound in self.boundary:
            theta = np.arctan((self.cg[1]-bound[1])/(bound[0]-self.cg[0]))
            # in 3rd and 4th quadrant // arctan (-pi/2, pi/2)
            if bound[0] < self.cg[0]:
                theta += np.pi
            elif bound[1] > self.cg[1]:
                theta += 2*np.pi
            theta_boundary.update({theta: bound})
        
        theta_boundary = dict(sorted(theta_boundary.items(), key=lambda item: item[0]))

        return theta_boundary
    
    def find_dist(self):
        theta_radial = {}
        for key, value in self.theta_dict.items():
            dist = np.sqrt(np.square(value[0]-self.cg[0]) + np.square(value[1]-self.cg[1]))
            theta_radial[key] = dist
        
        return theta_radial

    def plot(self):
        plt.imshow(self.image, cmap = 'gray')
        for i in self.boundary:
            plt.plot(i[0], i[1], marker = "o", markersize = 3, markeredgecolor = "red", markerfacecolor = "green")
            plt.plot(self.cg[0], self.cg[1], marker = "o", markersize = 1, markeredgecolor = "red", markerfacecolor = "green")
        plt.show()
    
    def plot_uniform(self, angle_degree):
        plt.imshow(self.image, cmap = 'gray')
        for key, value in self.theta_distance(angle_degree).items():
            plt.plot(self.cg[0]+value*np.cos(key), self.cg[1]-value*np.sin(key), marker = "o", markersize = 3, markeredgecolor = "red", markerfacecolor = "green")
            plt.plot(self.cg[0], self.cg[1], marker = "o", markersize = 1, markeredgecolor = "red", markerfacecolor = "green")
        plt.show()

    def plot_histogram(self, angle_degree):
        plt.figure(figsize = (8, 8))
        for key, value in self.theta_distance(angle_degree).items():
            plt.plot(key, value, marker = "o", markersize = 1, markeredgecolor = "red", markerfacecolor = "green")
        plt.xlim([0, max(list(self.theta_distance(angle_degree).values()))])
        plt.ylim([0, max(list(self.theta_distance(angle_degree).values()))])
        plt.show()

    def theta_distance(self, angle_degree):
        num = 360//angle_degree
        theta_dist = {}
        for i in range(num):
            angle_rad = np.pi / 180. * i * angle_degree
            
            if self.theta_radial.get(angle_rad):
                res = self.theta_radial.get(angle_rad)
            else:
                keys = list(self.theta_radial.keys())
                self.theta_radial[min(keys)+ 2*np.pi] = self.theta_radial[min(keys)]
                self.theta_radial[max(keys)- 2*np.pi] = self.theta_radial[max(keys)]
                self.theta_radial = dict(sorted(self.theta_radial.items(), key=lambda item: item[0]))
                
                nx_theta = min([i for i in list(self.theta_radial.keys()) if i>angle_rad])
                pv_theta = max([i for i in list(self.theta_radial.keys()) if i<angle_rad])
                
                next = self.theta_radial[nx_theta]
                prev = self.theta_radial[pv_theta]

                # Calculating the position of angle_rad between next and prev
                slope = (prev*np.sin(pv_theta) - next*np.sin(nx_theta)) / (prev*np.cos(pv_theta) - next*np.cos(nx_theta))
                temp = slope - np.tan(angle_rad)
                temp = 1/temp
                temp *= (prev*np.cos(pv_theta)*np.tan(angle_rad) - prev*np.sin(pv_theta))
                temp += prev*np.cos(pv_theta)
                res = temp / np.cos(angle_rad)

            theta_dist[angle_rad] = res
        
        return theta_dist

    
    ######

    def tumour_compactness(self):
        perimeter = len(self.boundary)
        area = np.count_nonzero(self.image > 0)

        return 1 - 4*np.pi*area/np.square(perimeter)

    ######

    def radial_dist_mean(self, angle_degree=1):
        total = 0
        for rad in self.theta_distance(angle_degree).values():
            total += rad
        
        return total / len(list(self.theta_distance(angle_degree).values()))

    def radial_dist_std(self, angle_degree=1):
        total = 0
        rdm = self.radial_dist_mean(angle_degree)
        for rad in self.theta_distance(angle_degree).values():
            total += np.square(rad - rdm)
        
        return total / len(list(self.theta_distance(angle_degree).values()))

    ######

    def area_ratio(self, angle_degree=1):
        total = 0
        rdm = self.radial_dist_mean(angle_degree)
        for rad in self.theta_distance(angle_degree).values():
            if rad > rdm:
                total += rad - rdm
        
        return total / rdm / len(list(self.theta_distance(angle_degree).values()))


In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask2.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)
distance = image1.find_dist()

theta_dict = image1.theta()

print('distance: ')
print(distance)

theta_dict = image1.theta()
print('\ntheta - coordinate:')
print(theta_dict)

print('\ncentroid: ', image1.centroid())
image1.plot()

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask2.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)
print(image1.theta_distance(60))
image1.plot_uniform(8)

image1.plot_histogram(1)

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask3.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)
distance = image1.find_dist()

theta_dict = image1.theta()

print('distance: ')
print(distance)

theta_dict = image1.theta()
print('\ntheta - coordinate:')
print(theta_dict)

print('\ncentroid: ', image1.centroid())
image1.plot()

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask3.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)
print(image1.theta_distance(60))
image1.plot_uniform(1)

image1.plot_histogram(1)

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask2.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)

image1.plot()

print(image1.tumour_compactness())


In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask3.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)

image1.plot()

print(image1.tumour_compactness())

Tumour compactness : 

benign -> 0

malignant -> 1

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask2.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)

image1.plot()

print(image1.radial_dist_mean(1))
print(image1.radial_dist_std(1))

In [ ]:
image1 = cv2.imread('/content/drive/MyDrive/project/mask3.png', 0)
image1 = image1/ 255.
image1 = np.where(image1 < 0.5, 0, 1)

image1 = mask_image(image1)

image1.plot()

print(image1.radial_dist_mean(1))
print(image1.radial_dist_std(1))

std of radial distance -> lesser for benign tumours